In [1]:
import networkx as nx

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 

# import community as community_louvain

from collections import Counter

import networkx.algorithms.community as nx_comm
import networkx.algorithms.flow as nx_flow
import networkx.algorithms.shortest_paths as nx_path

import collections
import itertools

In [2]:
studied_proteins = {
    'DIC1': '4932.YLR348C',
    'RGT2': '4932.YDL138W',
    'CBF5': '4932.YLR175W',
    'EST2': '4932.YLR318W'
}

In [3]:
# Generate common name for yeast proteins
temp_df = pd.read_table("4932.protein.info.v11.5.txt",delimiter='\t')
common_name = dict(temp_df.iloc[:,:2].values)

In [4]:
# Read in essential node data
essential_df = pd.read_csv('essential.csv', header=None)

In [5]:
# Read in preprocessed uniprot data
uniprot_df = pd.read_csv('uniprot_processed.csv')

In [6]:
# Read in dataframe for detailed protein links
G_dataframe = pd.read_table("4932.protein.links.detailed.v11.5.txt",delimiter=' ')

In [7]:
# Reweight edges of protein links
keep_columns = [
    'neighborhood', 'fusion', 'cooccurence', 
    'coexpression', 'experimental', 'database', 
    'textmining'
]

# Double weight of experimental evidence
G_dataframe['experimental'] *= 2

# Half weight of textmining
G_dataframe['textmining'] //=2

# Calculate combined score
G_dataframe.combined_score = G_dataframe[keep_columns].max(axis=1)

# Drop edges with combined score of zero
G_dataframe = G_dataframe.drop(G_dataframe[G_dataframe.combined_score == 0].index)

In [8]:
# Drop edges with combined score of <= threshold score
threshold_score = 600
G_dataframe = G_dataframe.drop(
    G_dataframe[G_dataframe.combined_score <= threshold_score].index
    )

In [9]:
# Sanity check of data

G_dataframe

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
67,4932.Q0045,4932.YML120C,0,0,0,0,0,900,55,900
71,4932.Q0045,4932.YOR257W,0,0,0,0,630,0,0,630
73,4932.Q0045,4932.YPL172C,155,0,370,499,630,0,419,630
75,4932.Q0045,4932.YGR033C,0,0,0,0,702,0,0,702
80,4932.Q0045,4932.YKL192C,0,0,0,49,1060,0,116,1060
...,...,...,...,...,...,...,...,...,...,...
1988527,4932.YPR203W,4932.YER189W,0,0,0,872,0,0,0,872
1988542,4932.YPR204W,4932.YPR018W,0,0,0,0,840,0,0,840
1988553,4932.YPR204W,4932.YBL032W,0,0,0,0,630,0,0,630
1988565,4932.YPR204W,4932.YNL255C,0,0,0,0,630,0,0,630


In [10]:
# Create nx graph

G = nx.from_pandas_edgelist(G_dataframe, 
    source='protein1', 
    target='protein2', 
    edge_attr=True)

In [11]:
# Ignore certain nodes in the protein network, and also combine uniprot data
# Remove proteins that are not in a location that we are targeting
# (we target membrane, mitochrondrial, cytoplasm and nucleus proteins)

irrelevant_proteins = []

# remove proteins that are ribosomal
ribosomal_proteins = []

row_index = {}

for node in G.nodes:
    # Get row index of matching row for a protein
    node_name = node.split('.')[1]
    node_matches = uniprot_df['Gene Names'].str.contains(node_name).fillna(False)
    node_uniprot = uniprot_df.index[node_matches][0]

    if uniprot_df['location_match'][node_uniprot] == 0:
        irrelevant_proteins.append(node)

    elif uniprot_df['ribosomal'][node_uniprot] == True:
        ribosomal_proteins.append(node)

    else:
        row_index[node] = node_uniprot

print(f"{len(irrelevant_proteins)=}, {len(ribosomal_proteins)=}")
print(f"{len(G.nodes)=}")
G.remove_nodes_from(irrelevant_proteins)
G.remove_nodes_from(ribosomal_proteins)
print(f"{len(G.nodes)=}")

In [ ]:
# Track location of nodes
node_location = {}

mitochondrial_proteins = set()
cytoplasm_proteins = set()
membrane_proteins = set()
nucleus_proteins = set()

for node in G.nodes:
    if uniprot_df['mitochon'][row_index[node]] == 1:
        node_location[node] = 'mitochondria'
        mitochondrial_proteins.add(node)
    elif uniprot_df['cytoplasm'][row_index[node]] == 1:
        node_location[node] = 'cytoplasm'
        cytoplasm_proteins.add(node)
    elif uniprot_df['membrane'][row_index[node]] == 1:
        node_location[node] = 'membrane'
        membrane_proteins.add(node)
    elif uniprot_df['nucleus'][row_index[node]] == 1:
        node_location[node] = 'nucleus'
        nucleus_proteins.add(node)
    else:
        node_location[node] = 'other'

nx.set_node_attributes(G, values=node_location, name='location')

In [ ]:
# We remove essential proteins, but keep proteins we want to study
essential_proteins_raw = essential_df[1].values
essential_proteins = set(map(lambda x: '4932.' + x, essential_proteins_raw))
essential_proteins -= set(studied_proteins.values())
print(f"{len(G.nodes)=}")
G.remove_nodes_from(list(essential_proteins))
print(f"{len(G.nodes)=}")

len(G.nodes)=4266
len(G.nodes)=3271


In [ ]:
# Let's only keep the largest connected component
largest_component = max(list(nx.connected_components(G)), key=len)
G = G.subgraph(largest_component)
print(f"{len(G.nodes)=}")

len(G.nodes)=3172


In [ ]:
# Sanity check - 
#   Check proteins we are interested in are still in the network

for key, value in studied_proteins.items():
    if value not in G.nodes:
        print(f"ERROR: {key} was deleted from the network!")

In [ ]:
direction = {
    'mitochondria': {'cytoplasm', 'mitochondria'},
    'cytoplasm': {'nucleus', 'mitochondria', 'cytoplasm'},
    'nucleus': {'nucleus'},
    'membrane': {'mitochondria', 'membrane'}
}

In [ ]:
GX = nx.DiGraph()
for edge in G.edges:
    if G.nodes[edge[1]]['location'] in direction[G.nodes[edge[0]]['location']]:
        GX.add_edge(edge[0], edge[1])
    if G.nodes[edge[0]]['location'] in direction[G.nodes[edge[1]]['location']]:
        GX.add_edge(edge[1], edge[0])

In [ ]:
for start in ['DIC1', 'RGT2']:
    for end in ['CBF5', 'EST2']:
        try:
            print('\n'.join(list(map(common_name.get, nx_path.shortest_path(GX, 
            studied_proteins[start], 
            studied_proteins[end])))),end='\n\n')
            print(len(list(nx_path.all_shortest_paths(GX, 
            studied_proteins[start], 
            studied_proteins[end]))))
        except:
            print('fail', start, end)

DIC1
PUF3
CBF5

1
DIC1
PUF3
HEK2
EST2

6
RGT2
GPR1
RAS2
PUF3
CBF5

4
RGT2
GPR1
RAS2
PUF3
HEK2
EST2

22


In [ ]:
def get_paths(start, end, func, min_count=1, **kwargs):
    paths = list(func(GX, 
            studied_proteins[start], 
            studied_proteins[end], **kwargs))
    all_occurrences = collections.Counter(itertools.chain.from_iterable(paths))
    return (p for p in paths if min(set(map(all_occurrences.get, p))) > min_count)

In [ ]:
def print_paths(start, end, func, reverse=False, **kwargs):
    out = lambda x: ', '.join(map(common_name.get, x))
    if reverse:
        out = lambda x: ', '.join(reversed(list(map(common_name.get, x))))
    print('\n'.join(sorted(map(out, get_paths(start, end, func, **kwargs)))))

In [ ]:
# All shortest paths
print_paths('RGT2', 'EST2', nx_path.all_shortest_paths, reverse=True)

EST2, CBF5, PUF3, HXT3, SNF3, RGT2
EST2, CBF5, PUF3, PEX13, SNF3, RGT2
EST2, CBF5, PUF3, RAS2, GPR1, RGT2
EST2, CBF5, PUF3, YOR1, BPT1, RGT2
EST2, HEK2, PUF3, HXT3, SNF3, RGT2
EST2, HEK2, PUF3, PEX13, SNF3, RGT2
EST2, HEK2, PUF3, RAS2, GPR1, RGT2
EST2, HEK2, PUF3, YOR1, BPT1, RGT2
EST2, NAB6, PUF3, HXT3, SNF3, RGT2
EST2, NAB6, PUF3, PEX13, SNF3, RGT2
EST2, NAB6, PUF3, RAS2, GPR1, RGT2
EST2, NAB6, PUF3, YOR1, BPT1, RGT2
EST2, PUB1, PUF3, HXT3, SNF3, RGT2
EST2, PUB1, PUF3, PEX13, SNF3, RGT2
EST2, PUB1, PUF3, RAS2, GPR1, RGT2
EST2, PUB1, PUF3, YOR1, BPT1, RGT2
EST2, TOR1, YPT7, YOR1, BPT1, RGT2


In [ ]:
# Number of simple paths with cutoff length 5
sum(map(lambda x: 1, nx.algorithms.simple_paths.all_simple_paths(GX, 
            studied_proteins['DIC1'], 
            studied_proteins['CBF5'], cutoff=5)))

79563

In [ ]:
# All simple paths with cutoff length 3
print_paths('DIC1', 'EST2', nx.algorithms.simple_paths.all_simple_paths, cutoff=5)

DIC1, ATP1, AAC1, CBF5, EBS1, EST2
DIC1, ATP1, AAC1, CBF5, EST2
DIC1, ATP1, AAC1, CBF5, HEK2, EST2
DIC1, ATP1, AAC1, CBF5, HUL4, EST2
DIC1, ATP1, AAC1, CBF5, LHP1, EST2
DIC1, ATP1, AAC1, CBF5, NOP12, EST2
DIC1, ATP1, AAC1, CYB2, HEK2, EST2
DIC1, ATP1, AAC1, GIS2, CBF5, EST2
DIC1, ATP1, AAC1, GIS2, DPB3, EST2
DIC1, ATP1, AAC1, GIS2, GBP2, EST2
DIC1, ATP1, AAC1, GIS2, HRB1, EST2
DIC1, ATP1, AAC1, GIS2, LHP1, EST2
DIC1, ATP1, AAC1, GIS2, NMD2, EST2
DIC1, ATP1, AAC1, PHB1, TOR1, EST2
DIC1, ATP1, AAC1, PIC2, HEK2, EST2
DIC1, ATP1, AAC3, CBF5, EBS1, EST2
DIC1, ATP1, AAC3, CBF5, EST2
DIC1, ATP1, AAC3, CBF5, HEK2, EST2
DIC1, ATP1, AAC3, CBF5, HUL4, EST2
DIC1, ATP1, AAC3, CBF5, LHP1, EST2
DIC1, ATP1, AAC3, CBF5, NOP12, EST2
DIC1, ATP1, AAC3, KAP123, CBF5, EST2
DIC1, ATP1, AAC3, KAP123, EST1, EST2
DIC1, ATP1, AAC3, KAP123, ISW1, EST2
DIC1, ATP1, AAC3, KAP123, NOP12, EST2
DIC1, ATP1, AAC3, PHB1, TOR1, EST2
DIC1, ATP1, AAC3, PIC2, HEK2, EST2
DIC1, ATP1, AAC3, RAD5, ISW1, EST2
DIC1, ATP1, AAC3, RAD

In [54]:
cut = 5
# paths = nx.algorithms.simple_paths.all_simple_paths(GX, 
#             studied_proteins['DIC1'], 
#             studied_proteins['CBF5'], cutoff=cut)
allpaths = list(get_paths('DIC1', 'CBF5', nx.algorithms.simple_paths.all_simple_paths, cutoff=5, min_count=1500))
#allpaths = [np.array(p) for p in paths]
#pathlist = list(paths)
#allpaths
# allpathsets = [set(zip(p[0:-1], p[1:])) for p in paths]

In [55]:
len(allpaths)

10425

In [43]:
def path_similarity1(p,q):
    try:
        return (sum(p == q) -2)
    except:
        minlen = min(len(p), len(q))
        return (sum(p[:minlen] == q[:minlen]))

In [44]:
def path_similarity2(p,q):
    p_len = len(p)-1
    q_len = len(q)-1
    
    P = [(p[k], p[k+1]) for k in range(p_len)]
    Q = [(q[k], q[k+1]) for k in range(q_len)]
    
    return len(set(P) & set(Q)) / len(set(P) | set(Q))

In [45]:
def path_similarity3(p,q):
    P = set(zip(p[0:-1], p[1:]))
    Q = set(zip(q[0:-1], q[1:]))
    
    return len(P & Q) / len(P | Q)

In [46]:
def path_similarity4(p,q):
    P = set(p)
    Q = set(q)
    return len(P & Q) / len(P | Q)

In [47]:
# allpathsets

In [ ]:
(len(set(allpaths[6]) | set(allpaths[1]))-2)

6

In [ ]:
list(enumerate(allpaths[10:], start = 10))

[(10,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBL099W', '4932.YGL178W',
         '4932.YLR175W'], dtype='<U12')),
 (11,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBL099W', '4932.YMR056C',
         '4932.YLR175W'], dtype='<U12')),
 (12,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBL099W', '4932.YOR276W',
         '4932.YLR175W'], dtype='<U12')),
 (13,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBR039W', '4932.YNL255C',
         '4932.YLR175W'], dtype='<U12')),
 (14,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBR039W', '4932.YLL013C',
         '4932.YLR175W'], dtype='<U12')),
 (15,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBR039W', '4932.YOR187W',
         '4932.YLR175W'], dtype='<U12')),
 (16,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBR221C', '4932.YLL013C',
         '4932.YLR175W'], dtype='<U12')),
 (17,
  array(['4932.YLR348C', '4932.YEL024W', '4932.YBR221C', '4932.YGL178W',
         '4932.YLR175W'], dtype='<U12')),
 (18,
  array(['4932.YLR348C', '

In [51]:
a = nx.complete_graph(map(tuple, allpaths))

KeyboardInterrupt: 

In [56]:
HomotopyG = nx.Graph()
for i, p in enumerate(allpaths):
    for j, q in enumerate(allpaths[i:], start = i):
        sim = path_similarity4(p,q)
        if sim > 0:
            HomotopyG.add_edge(i,j)
    if i% 100 == 0:
        print(i)

0
100
200
300
400
500
600
700


: 

: 

In [ ]:
HomotopyG = nx.Graph()
for i, p in enumerate(pathlist):
    for j, q in enumerate(pathlist[i:], start = i):
        sim = path_similarity2(p,q)
        if sim > 0:
            HomotopyG.add_edge(i,j, weight=sim)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


In [ ]:
HomotopyG = nx.Graph()
for i, p in enumerate(pathlist):
    for j, q in enumerate(pathlist[i:], start = i):
        if len(set(p[1:-1]) & set(q[1:-1])) > 0:
            HomotopyG.add_edge(i,j, weight=path_similarity3(p,q))
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


In [ ]:
HomotopyG = nx.Graph()
for i, P in enumerate(allpathsets):
    for j, Q in enumerate(allpathsets[i:], start = i):
        HomotopyG.add_edge(i,j, weight = len(P & Q) / len(P | Q))
    if i % 100 == 0:
        print(i)

0
100
200


KeyboardInterrupt: 

In [ ]:
['a','b','c'][1:-1]

['b']

In [ ]:
len(HomotopyG.nodes)

2107

In [ ]:
Branches = community_louvain.best_partition(HomotopyG, resolution=1)

KeyboardInterrupt: 

In [ ]:
Branches

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122: 0,
 12